# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/model/')

In [2]:
import pandas as pd
import numpy as np
import pickle
import itertools
import random
import torch
from model.utils import *

In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Data

In [5]:
with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(train_dataset_dict_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)

with open(test_dataset_dict_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [6]:
Lr

0.09138694552832571

# Hierarchical GRU

In [7]:
def hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path):
    hgru_models = {}

    num_categories = 0
    for indent in sorted(list(categories_per_indent_dict.keys())):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]:
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                loss_coef = 4.0852794322516647*np.exp(-7) #post optuna
                parent_model = Model
                parent_optimizer = Optimizer
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = Model
            optimizer = Optimizer
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            training_and_evaluation(model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)


In [8]:
hgru_models = hgru_model(son_parent_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath)

num categories: 1
category id|name: 2|All-items
num categories: 2
category id|name: 256|Alcoholic beverages, tobacco products and recreational cannabis
num categories: 3
category id|name: 290|All-items excluding alcoholic beverages, tobacco products and smokers' supplies and recreational cannabis
num categories: 4
category id|name: 287|All-items excluding energy
num categories: 5
category id|name: 284|All-items excluding food
num categories: 6
category id|name: 285|All-items excluding food and energy
num categories: 7
category id|name: 302|All-items excluding gasoline
num categories: 8
category id|name: 289|All-items excluding mortgage interest cost
num categories: 9
category id|name: 293|All-items excluding shelter
num categories: 10
category id|name: 139|Clothing and footwear
num categories: 11
category id|name: 288|Energy
num categories: 12
category id|name: 3|Food
num categories: 13
category id|name: 286|Food and energy
num categories: 14
category id|name: 201|Health and personal c

In [9]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model = Model
        basic_optimizer = Optimizer
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [10]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [11]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

In [12]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Get Best Model Weights:

In [13]:
#def get_weights_per_category(category_list, dir_path):
#    weights = {}
#    for category in category_list:
#        model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
#        model.to(device)
#        optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
        
#        best_checkpoint_path = dir_path+category + '.pt'

#        category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(best_checkpoint_path, model, optimizer)
#        category_model_weights = unify_model_weights(category_model)

#        weights[category] = category_model_weights
    
#    return weights

In [14]:
def get_weights_per_category(category_id_list, dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    #basic_model.to(device)

    best_models_weights_dict = {}

    for category_id in category_id_list:
        category_name = category_id_to_name_dict[category_id]
        ckp_path = dir_path+category_name+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        category_model_weights = unify_model_weights(model)
        best_models_weights_dict[category_id] = category_model_weights
        
    return best_models_weights_dict

In [15]:
dir_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_canada/models_weights/'

In [16]:
category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])


In [17]:
len(categories)

293

In [18]:
len(category_id_list)

293

In [19]:
weights_dict = get_weights_per_category(category_id_list, dir_path)

In [ ]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/hgru_model_weights_canada.pickle', 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)